In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1629139006630_0002,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [12]:
censo = (
    spark
    .read
    .format("csv")
    .option("header", True)
    .option("inferSchema", True)
    .option("delimiter", "|")
    .load("s3://mindsetcloud/raw-data/")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [13]:
censo.printSchema()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

root
 |-- NU_ANO_CENSO: integer (nullable = true)
 |-- ID_ALUNO: string (nullable = true)
 |-- ID_MATRICULA: integer (nullable = true)
 |-- NU_MES: integer (nullable = true)
 |-- NU_ANO: integer (nullable = true)
 |-- NU_IDADE_REFERENCIA: integer (nullable = true)
 |-- NU_IDADE: integer (nullable = true)
 |-- TP_SEXO: integer (nullable = true)
 |-- TP_COR_RACA: integer (nullable = true)
 |-- TP_NACIONALIDADE: integer (nullable = true)
 |-- CO_PAIS_ORIGEM: integer (nullable = true)
 |-- CO_UF_NASC: integer (nullable = true)
 |-- CO_MUNICIPIO_NASC: integer (nullable = true)
 |-- CO_UF_END: integer (nullable = true)
 |-- CO_MUNICIPIO_END: integer (nullable = true)
 |-- TP_ZONA_RESIDENCIAL: integer (nullable = true)
 |-- TP_LOCAL_RESID_DIFERENCIADA: integer (nullable = true)
 |-- IN_NECESSIDADE_ESPECIAL: integer (nullable = true)
 |-- IN_BAIXA_VISAO: integer (nullable = true)
 |-- IN_CEGUEIRA: integer (nullable = true)
 |-- IN_DEF_AUDITIVA: integer (nullable = true)
 |-- IN_DEF_FISICA: int

In [22]:
'''(
    censo
    .write
    .mode("overwrite")
    .format("parquet")
    .partitionBy("NU_ANO_CENSO")
    .save("s3://mindsetcloud-production-386643807537/staging/")
)'''

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [3]:
censo = (
    spark
    .read
    .format("parquet")
    .option("header", True)
    .option("inferSchema", True)
    .load("s3://mindsetcloud-production-386643807537/staging/")
)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
from pyspark.sql.functions import mean, max, min, col, count, sum

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [40]:
#Qual foi o estado brasileiro (CO_UF) com o segundo maior número de alunos no Censo Escolar 2020?
(censo.groupby('CO_UF').agg(count('ID_MATRICULA').alias('TOTAL_MATRICULADOS')).sort(col("TOTAL_MATRICULADOS").desc()).show(truncate=False))


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+------------------+
|CO_UF|TOTAL_MATRICULADOS|
+-----+------------------+
|35   |10399931          |
|31   |4494458           |
|33   |3617974           |
|29   |3507272           |
|41   |2731721           |
|23   |2377672           |
|43   |2377005           |
|15   |2309261           |
|26   |2248002           |
|21   |2094724           |
|42   |1716964           |
|52   |1509671           |
|13   |1173846           |
|25   |977977            |
|32   |915822            |
|22   |882072            |
|51   |871484            |
|27   |867722            |
|24   |826043            |
|53   |724465            |
+-----+------------------+
only showing top 20 rows

In [21]:
#No estado de Minas Gerais, quantos alunos moravam na zona rural (tp_zona_residencial = 2)?
censo.filter(censo.CO_UF == 31 ).groupby('TP_ZONA_RESIDENCIAL').agg(count('ID_MATRICULA')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|TP_ZONA_RESIDENCIAL|count(ID_MATRICULA)|
+-------------------+-------------------+
|                  1|            3938404|
|                  2|             556054|
+-------------------+-------------------+

In [22]:
#No estado de São Paulo, quantos alunos moravam na zona urbana (tp_zona_residencial = 1)?
censo.filter(censo.CO_UF == 35 ).groupby('TP_ZONA_RESIDENCIAL').agg(count('ID_MATRICULA')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------------------+-------------------+
|TP_ZONA_RESIDENCIAL|count(ID_MATRICULA)|
+-------------------+-------------------+
|                  1|            9905267|
|                  2|             494664|
+-------------------+-------------------+

In [33]:
#No estado do Espírito Santo, quantos alunos do sexo masculino (tp_sexo=1) tinham autismo?
censo.filter((censo.CO_UF == 32) & (censo.IN_AUTISMO == 1)).groupby('TP_SEXO').agg(count('IN_AUTISMO')).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-------+-----------------+
|TP_SEXO|count(IN_AUTISMO)|
+-------+-----------------+
|      1|             9326|
|      2|             2322|
+-------+-----------------+

In [43]:
#Qual estado brasileiro possui o segundo maior número de alunas do sexo feminino com cegueira (in_cegueira=1)?
censo.filter((censo.TP_SEXO == 2) & (censo.IN_CEGUEIRA == 1)).groupby('CO_UF').agg(count('ID_MATRICULA')).sort(col("count(ID_MATRICULA)").desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+
|CO_UF|count(ID_MATRICULA)|
+-----+-------------------+
|   35|                718|
|   33|                523|
|   31|                401|
|   29|                395|
|   43|                335|
|   41|                293|
|   15|                292|
|   21|                206|
|   52|                197|
|   23|                194|
|   26|                171|
|   42|                143|
|   25|                141|
|   24|                121|
|   51|                 90|
|   22|                 88|
|   53|                 87|
|   32|                 86|
|   13|                 84|
|   50|                 83|
+-----+-------------------+
only showing top 20 rows

In [44]:
#Qual é a diferença (em número absoluto de pessoas) entre o segundo e o terceiro estados brasileiros que possuem os maiores números de alunas do sexo feminino com surdez (in_surdez=1)?
censo.filter((censo.TP_SEXO == 2) & (censo.IN_SURDEZ == 1)).groupby('CO_UF').agg(count('ID_MATRICULA')).sort(col("count(ID_MATRICULA)").desc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+
|CO_UF|count(ID_MATRICULA)|
+-----+-------------------+
|   35|               3007|
|   31|               1117|
|   41|               1012|
|   29|                962|
|   15|                870|
|   26|                780|
|   43|                695|
|   23|                665|
|   21|                618|
|   33|                585|
|   52|                531|
|   42|                523|
|   25|                388|
|   13|                378|
|   27|                336|
|   24|                305|
|   22|                270|
|   32|                246|
|   51|                201|
|   50|                192|
+-----+-------------------+
only showing top 20 rows

In [45]:
#Qual é o estado brasileiro que possui o menor número de alunos matriculados no 9 ano (tp_etapa_ensino = 41)?
censo.filter((censo.TP_ETAPA_ENSINO == 41) ).groupby('CO_UF').agg(count('ID_MATRICULA')).sort(col("count(ID_MATRICULA)").asc()).show()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+-----+-------------------+
|CO_UF|count(ID_MATRICULA)|
+-----+-------------------+
|   14|              10327|
|   16|              11517|
|   12|              14720|
|   17|              24929|
|   11|              25961|
|   28|              29053|
|   50|              36966|
|   24|              40235|
|   53|              42052|
|   22|              44331|
|   27|              45269|
|   32|              47447|
|   25|              51321|
|   51|              51948|
|   13|              65816|
|   42|              89842|
|   52|              94122|
|   21|             111014|
|   23|             122388|
|   43|             125677|
+-----+-------------------+
only showing top 20 rows

In [ ]:
#censo.select(mean('NU_ANO_CENSO')).show()